## Deep Learning Project Sapienza


# Task
Question Answering on Mathematics Dataset Mathematical reasoning – a
core ability within human intelligence – presents some unique challenges as a domain:
we do not come to understand and solve mathematical problems primarily on the back
of experience and evidence, but on the basis of inferring, learning, and exploiting laws,
axioms, and symbol manipulation rules. The task consists of many different types
of mathematics problems, with the motivation that it should be harder for a model
to do well across a range of problem types without possessing at least some part of
these abilities that allow for algebraic generalization.

Questo vecchio foglio colab ha in realta ottenuto dei risultati migliori del finale e abbiamo quindi deciso di aggiungerlo.


## imports

In [ ]:
# @title
!pip install wget pytorch_lightning datasets transformers --quiet

In [ ]:
# @title
!pip install --upgrade gdown --quiet

In [ ]:
# @title
# Importing dependencies
import json
import os
import wget
import random
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2Tokenizer
from transformers import BertTokenizer
from torch.utils.data import random_split, DataLoader
from pytorch_lightning import Trainer, LightningDataModule, LightningModule
from datasets import load_dataset

In [ ]:
# @title
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def answers_same_length(answers_tokenized, pad_value, max_length):
  for i in range(0, len(answers_tokenized['input_ids'])):
    if len(answers_tokenized['input_ids'][i]) < max_length:
      [answers_tokenized['input_ids'][i].append(pad_value) for _ in range(0, (max_length - len(answers_tokenized['input_ids'][i])))]
      [answers_tokenized['attention_mask'][i].append(0) for _ in range(0, (max_length - len(answers_tokenized['attention_mask'][i])))]

  return answers_tokenized

In [ ]:
class Dataset_class:
  def __init__(self, inputs):
    self.input_ids = inputs['inputs']['input_ids']
    self.attention_mask = inputs['inputs']['attention_mask']
    self.answer_ids = inputs['outputs']['input_ids']
    self.answer_attention_mask = inputs['outputs']['attention_mask']

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, item):
    return [torch.tensor(self.input_ids[item], dtype=torch.long).to(device),
            torch.tensor(self.attention_mask[item], dtype=torch.long).to(device),
            torch.tensor(self.answer_ids[item], dtype=torch.long).to(device),
            torch.tensor(self.answer_attention_mask[item], dtype=torch.long).to(device)]

In [ ]:
def paper_accuracy(predicted, target):

  predicted = torch.argmax(predicted, dim=2)
  #print("prediction pre: ", predicted.shape)
  score_per_char = 0
  total_score = 0
  count = 0
  size_0 = predicted.size(0)
  size_1 = predicted.size(1)


  for i in range(0, size_0):
    score = 0
    for j in range(0, size_1):
      if (predicted[i][j].item() == target[i][j].item()):
        score += 1
      count += 1
      if target[i][j].item() == 102:
        break
    if score == count:
      total_score += 1
    score_per_char += score

    total_accuracy = total_score / size_0
    total_acc_char = score_per_char / count#(size_0 * size_1)



  return total_accuracy, total_acc_char

## preproccessing

In [ ]:
mathdataset = load_dataset('math_dataset', 'algebra__linear_1d')
mathdataset.set_format(type="torch")
mathdataset.save_to_disk(f"/content/drive/MyDrive/mathdataset_saved")


#

In [8]:
from datasets import load_from_disk
mathdataset = load_from_disk(f"/content/drive/MyDrive/mathdataset_saved")

In [9]:
n_train_rows = 10000
n_test_rows = 1000
from datasets import DatasetDict

mathdataset = DatasetDict({
    "train": mathdataset["train"].shuffle(seed=42).select(range(n_train_rows)),
    "test": mathdataset["test"].shuffle(seed=42).select(range(n_test_rows))
})



In [10]:
# Initialize empty dictionaries
char_to_idx = {}
idx_to_char = {}

# Combine characters from both "train" and "test" splits
all_characters = set()
question_max_length=0
answer_max_length=0
for split_name in ["train", "test"]:
    split_dataset = mathdataset[split_name]

    # Loop through the split dataset
    for row in split_dataset:
        question = row['question']
        answer = row['answer']
        question_length = len(row['question'])
        answer_length = len(row['answer'])
        # Include all unique characters from questions and answers
        all_characters.update(question)
        all_characters.update(answer)
        if question_length > question_max_length:
            question_max_length = question_length

        if answer_length > answer_max_length:
            answer_max_length = answer_length

# Assign each character a unique index
for idx, char in enumerate(all_characters):
    char_to_idx[char] = idx
    idx_to_char[idx] = char

'''#add special tokens
char_to_idx['<start>'] = len(char_to_idx)
char_to_idx['<end>'] = len(char_to_idx)
char_to_idx['PAD'] = len(char_to_idx)
idx_to_char[char_to_idx['<start>']] = '<start>'
idx_to_char[char_to_idx['<end>']] = '<end>'
idx_to_char[char_to_idx['PAD']] = 'PAD'''

# Now you have char_to_idx and idx_to_char dictionaries
print("Question Max Length:", question_max_length)
print("Answer Max Length:", answer_max_length)

Question Max Length: 66
Answer Max Length: 9


In [11]:
# Initialize empty dictionaries
char_to_idx = {}
idx_to_char = {}

# Combine characters from both "train" and "test" splits
all_characters = set()
question_max_length = 0
answer_max_length = 0

for split_name in ["train", "test"]:
    split_dataset = mathdataset[split_name]

    # Loop through the split dataset
    for row in split_dataset:
        question = row['question']
        answer = row['answer']
        question_length = len(row['question'])
        answer_length = len(row['answer'])

        # Include all unique characters from questions and answers
        all_characters.update(question)
        all_characters.update(answer)

        if question_length > question_max_length:
            question_max_length = question_length

        if answer_length > answer_max_length:
            answer_max_length = answer_length

# Assign each character a unique index
char_idx = 0  # Initialize a variable to keep track of character indices

for char in all_characters:
    char_to_idx[char] = char_idx
    idx_to_char[char_idx] = char
    char_idx += 1  # Increment the index for the next character

# Add special tokens
special_tokens = ['<start>', '<end>', 'PAD']

for token in special_tokens:
    char_to_idx[token] = char_idx
    idx_to_char[char_idx] = token
    char_idx += 1  # Increment the index for the next special token


In [12]:
# Set a fixed sequence length (maximum length of any string)
string_list = mathdataset["test"]["question"]
max_sequence_length =question_max_length + 2
# Initialize an empty list to hold the padded sequences
padded_sequences = []

# Convert the list of strings to a list of numerical indices, padding if necessary
for string in string_list:
    '''string = '<start>' + string + '<end>' '''
    numerical_indices = [char_to_idx[char] if char in char_to_idx else char_to_idx[' '] for char in string] #PAD
    if len(numerical_indices) < max_sequence_length:
        # Pad the sequence with space (or any other padding value)
        numerical_indices += [char_to_idx[' ']] * (max_sequence_length - len(numerical_indices)) #PAD
    padded_sequences.append(numerical_indices)

# Create a tensor from the list of padded numerical indices
testquestion = torch.tensor(padded_sequences)


NameError: ignored

In [ ]:
# Set a fixed sequence length (maximum length of any string)
string_list = mathdataset["test"]["answer"]
max_sequence_length =answer_max_length
# Initialize an empty list to hold the padded sequences
padded_sequences = []

# Convert the list of strings to a list of numerical indices, padding if necessary
for string in string_list:
    numerical_indices = [char_to_idx[char] if char in char_to_idx else char_to_idx[' '] for char in string]
    if len(numerical_indices) < max_sequence_length:
        # Pad the sequence with space (or any other padding value)
        numerical_indices += [char_to_idx[' ']] * (max_sequence_length - len(numerical_indices))
    padded_sequences.append(numerical_indices)

# Create a tensor from the list of padded numerical indices
testanswer = torch.tensor(padded_sequences)


In [ ]:
# Set a fixed sequence length (maximum length of any string)
string_list = mathdataset["train"]["answer"]
max_sequence_length =answer_max_length
# Initialize an empty list to hold the padded sequences
padded_sequences = []

# Convert the list of strings to a list of numerical indices, padding if necessary
for string in string_list:
    numerical_indices = [char_to_idx[char] if char in char_to_idx else char_to_idx[' '] for char in string]
    if len(numerical_indices) < max_sequence_length:
        # Pad the sequence with space (or any other padding value)
        numerical_indices += [char_to_idx[' ']] * (max_sequence_length - len(numerical_indices))
    padded_sequences.append(numerical_indices)

# Create a tensor from the list of padded numerical indices
trainanswer = torch.tensor(padded_sequences)


In [ ]:
# Set a fixed sequence length (maximum length of any string)
string_list = mathdataset["train"]["question"]
max_sequence_length =question_max_length
# Initialize an empty list to hold the padded sequences
padded_sequences = []

# Convert the list of strings to a list of numerical indices, padding if necessary
for string in string_list:
    numerical_indices = [char_to_idx[char] if char in char_to_idx else char_to_idx[' '] for char in string]
    if len(numerical_indices) < max_sequence_length:
        # Pad the sequence with space (or any other padding value)
        numerical_indices += [char_to_idx[' ']] * (max_sequence_length - len(numerical_indices))
    padded_sequences.append(numerical_indices)

# Create a tensor from the list of padded numerical indices
trainquestion = torch.tensor(padded_sequences)


In [ ]:
# Reduce the dataset to train it
#n = int(1000)
#p = int(n/5)

#trainquestion = trainquestion[1:n, :68]

#testanswer = testanswer[1:p, :68]

#testquestion = testquestion[1:p, :68]

#trainanswer = trainanswer[1:n, :68]


In [ ]:

import numpy as np
def create_attention_mask(input_tensor):
    """
    Create a fully connected attention mask for a given tensor.

    Args:
        input_tensor (numpy.ndarray): The input tensor.

    Returns:
        numpy.ndarray: The attention mask with 1s indicating positions to attend to.
    """
    N, M = input_tensor.shape
    attention_mask = np.ones((N, M), dtype=int)
    return attention_mask

In [ ]:
class Dataset_class2:
  def __init__(self, inputs1,inputs2,inputs3,inputs4):
    self.input_ids = inputs1
    self.attention_mask = inputs2
    self.answer_ids = inputs3
    self.answer_attention_mask = inputs4

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, item):
    return [torch.tensor(self.input_ids[item], dtype=torch.long).to(device),
            torch.tensor(self.attention_mask[item], dtype=torch.long).to(device),
            torch.tensor(self.answer_ids[item], dtype=torch.long).to(device),
            torch.tensor(self.answer_attention_mask[item], dtype=torch.long).to(device)]

In [ ]:
traindataset = Dataset_class2(trainquestion,create_attention_mask(trainquestion),trainanswer,create_attention_mask(trainanswer))
train_dataloaders = torch.utils.data.DataLoader(traindataset, shuffle = True, batch_size = 16)
testdataset = Dataset_class2(testquestion,create_attention_mask(testquestion),testanswer,create_attention_mask(testanswer))
test_dataloaders = torch.utils.data.DataLoader(testdataset, shuffle = True, batch_size = 16)


## Lstm model

In [73]:
class LSTMmodel(LightningModule):
  def __init__(self, char_to_idx, idx_to_char, embedding_size=512, hidden_units=2048, num_heads=8,
                num_encoder_layers=6, num_decoder_layers=6, padding_idx=0, p=0.):
      super().__init__()

      self.pad_token = "PAD"
      self.dict_size = len(char_to_idx)
      self.question_max_length = 68
      self.answer_max_length = 9
      self.padding_idx = padding_idx
      self.idx_to_char = idx_to_char
      self.dropout_probability = p
      self.batch_size = 128
      self.embedding_size = embedding_size
      self.hidden_units = hidden_units
      self.num_encoder_layers = num_encoder_layers  # Add this line

      # Define the LSTM layers
      self.embedding = nn.Embedding(self.dict_size, self.embedding_size, padding_idx=self.padding_idx)
      self.lstm = nn.LSTM(input_size=self.embedding_size, hidden_size=self.hidden_units,
                          num_layers=self.num_encoder_layers, dropout=self.dropout_probability, batch_first=True)
      self.output_layer = nn.Linear(self.hidden_units, self.dict_size)
  def forward(self, batch):
      # Assuming batch is a tuple of tensors
      input_tensor = batch[0]  # Access the input tensor from the tuple

      # Embed the input batch
      embedded = self.embedding(input_tensor)

      # Initialize the hidden state and cell state
      h0 = torch.zeros(self.num_encoder_layers, embedded.size(0), self.hidden_units).to(embedded.device)
      c0 = torch.zeros(self.num_encoder_layers, embedded.size(0), self.hidden_units).to(embedded.device)

      # Forward pass through the LSTM layers
      lstm_out, _ = self.lstm(embedded, (h0, c0))

      # Determine the sequence length based on batch[2]
      sequence_length = batch[2].size(1)

      # Reshape the LSTM output to (batch_size, sequence_length, hidden_units)
      lstm_out = lstm_out[:, :sequence_length, :]

      # Pass through the output layer to get logits
      logits = self.output_layer(lstm_out)

      return logits
  def training_step(self, batch, _):
    # Preparing inputs
    batch_answers = batch[2][:, 1:].flatten(0, 1)  # shape (batch_size * answer_max_length-1)
    batch[2] = batch[2][:, :-1]  # shape (batch_size, answer_max_length-1)

    # Computing prediction and loss
    pred = self(batch).flatten(0, 1)  # shape (batch_size * answer_max_length-1, dict_size)

    loss = F.cross_entropy(pred, batch_answers)
    print("loss: ", loss)
    return loss

  def validation_step(self, batch, _):
    #input_ids, _, batch_answers, _ = batch
    batch_answers = batch[2]  # shape (batch_size, answer_max_length)

    # Computing prediction and accuracy
    pred = self.predict(batch)  # shape (batch_size, answer_max_length-1, dict_size)
    accuracy = paper_accuracy(pred, batch_answers)  # accuracy for the current batch as defined in the "mathematics dataset" paper

    print("val_tot_accuracy", accuracy)  # at the end of every epoch it logs the average of the accuracies of each batch
    return accuracy

  def test_step(self, batch, batch_idx):
    batch_answers = batch[2]  # shape (batch_size, answer_max_length)

    pred = self.predict(batch.copy())  # shape (batch_size, answer_max_length-1, dict_size)
    accuracy = paper_accuracy(pred, batch_answers)  # accuracy for the current batch as defined in the "mathematics dataset" paper

    print("test_tot_accuracy", accuracy)  # at the end of every epoch it is logged the average of the accuracies of each batch
    return accuracy

  def predict(self, batch):  # shape (batch_size, question_max_length)
    batch[2] = torch.tensor([[1] for j in range(len(batch[2]))], device=device).long()

    for i in range(self.answer_max_length-1):
      transformer_result = self(batch)
      batch[2] = torch.tensor([], device=device).long()

      for j in range(len(transformer_result)):
        predicted_chars = torch.argmax(transformer_result[j], 1)

        if len(predicted_chars.shape)==1:
          predicted_chars = predicted_chars.unsqueeze(0)

        start_line_char = torch.tensor([1], device=device).unsqueeze(0).long()
        predicted_chars = torch.cat((start_line_char, predicted_chars), 1)

        batch[2] = torch.cat((batch[2], predicted_chars), 0)

    return F.one_hot(batch[2], num_classes=self.dict_size)  # shape (batch_size, answer_max_length-1, dict_size)

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=1e-4, betas=(0.9, 0.995))



In [74]:
def paper_accuracy(predicted, target):
  print("prediction pre: ", predicted.shape)
  print("target pre: ", target.shape)
  predicted = torch.argmax(predicted, dim=2)
  #print("prediction pre: ", predicted.shape)
  score_per_char = 0
  total_score = 0
  count = 0
  size_0 = predicted.size(0)
  size_1 = predicted.size(1)
  print("prediction: ", predicted.shape)
  print("target: ", target.shape)

  for i in range(0, size_0):
    score = 0
    for j in range(0, size_1):
      if (predicted[i][j].item() == target[i][j].item()):
        score += 1
      count += 1
      if target[i][j].item() == 102:
        break
    if score == count:
      total_score += 1
    score_per_char += score

    total_accuracy = total_score / size_0
    total_acc_char = score_per_char / count#(size_0 * size_1)

  for i in range(0, size_0):
    print("predicted: ", predicted[i])
    print("target: ", target[i])

  return total_accuracy, total_acc_char

In [75]:
# Initialize the model
model = LSTMmodel(char_to_idx, idx_to_char)

In [76]:
import pytorch_lightning as pl
trainer = pl.Trainer(
                  # Set the number of GPUs to use (change to 0 if you want to run on CPU)
    max_epochs=1,   # Number of updates per epoch to display
)
# Train the model using the train_dataloaders
trainer.fit(model, train_dataloaders,test_dataloaders)

# Test the model using the test_dataloaders
trainer.test(model, test_dataloaders)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type      | Params
-------------------------------------------
0 | embedding    | Embedding | 24.6 K
1 | lstm         | LSTM      | 188 M 
2 | output_layer | Linear    | 98.4 K
-------------------------------------------
188 M     Trainable params
0         Non-trainable params
188 M     Total params
755.860   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.
<ipython-input-55-2563b7be2ee6>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return [torch.tensor(self.input_ids[item], dtype=torch.long).to(device),
<ipython-input-55-2563b7be2ee6>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone

prediction pre:  torch.Size([16, 9, 48])
target pre:  torch.Size([16, 9])
prediction:  torch.Size([16, 9])
target:  torch.Size([16, 9])
predicted:  tensor([ 1, 28, 28, 28, 28, 28, 28, 28, 28], device='cuda:0')
target:  tensor([ 3, 40, 36,  6, 33, 40,  0,  0,  0], device='cuda:0')
predicted:  tensor([ 1, 28, 28, 28, 28, 28, 28, 28, 28], device='cuda:0')
target:  tensor([ 3, 40,  8,  8,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([ 1, 28, 28, 28, 28, 28, 28, 28, 28], device='cuda:0')
target:  tensor([ 3, 40, 31,  6, 33, 40,  0,  0,  0], device='cuda:0')
predicted:  tensor([ 1, 28, 28, 28, 28, 28, 28, 28, 28], device='cuda:0')
target:  tensor([ 3, 40, 23, 43, 36,  6, 33, 40,  0], device='cuda:0')
predicted:  tensor([ 1, 28, 28, 28, 28, 28, 28, 28, 28], device='cuda:0')
target:  tensor([ 3, 40, 14, 27,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([ 1, 28, 28, 28, 28, 28, 28, 28, 28], device='cuda:0')
target:  tensor([ 3, 40, 23,  8, 30,  6, 33, 40,  0], device='cud

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


...

prediction pre:  torch.Size([16, 9, 48])
target pre:  torch.Size([16, 9])
prediction:  torch.Size([16, 9])
target:  torch.Size([16, 9])
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 23, 36, 27,  6, 33, 40,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 23, 43, 31,  6, 33, 40,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40,  8, 14,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 23,  8, 31,  6, 33, 40,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 23,  8, 10,  6, 33, 40,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 23, 36,  8,  6, 33, 40,  0], device='cud

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

prediction pre:  torch.Size([16, 9, 48])
target pre:  torch.Size([16, 9])
prediction:  torch.Size([16, 9])
target:  torch.Size([16, 9])
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 43, 10,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 31, 36,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 23, 43,  8,  6, 33, 40,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 23,  8, 14,  6, 33, 40,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 23, 14,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40,  8,  9,  6, 33, 40,  0,  0], device='cud

[{}]

In [77]:
trainer.test(model, test_dataloaders)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

<ipython-input-55-2563b7be2ee6>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return [torch.tensor(self.input_ids[item], dtype=torch.long).to(device),
<ipython-input-55-2563b7be2ee6>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(self.answer_ids[item], dtype=torch.long).to(device),


prediction pre:  torch.Size([16, 9, 48])
target pre:  torch.Size([16, 9])
prediction:  torch.Size([16, 9])
target:  torch.Size([16, 9])
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 36, 27,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 43,  6, 33, 40,  0,  0,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 23, 31, 43,  6, 33, 40,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 36, 10,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 43, 43,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 23, 43, 43,  6, 33, 40,  0], device='cud

In [78]:
import torch

# Assuming 'model' is your PyTorch model
# Save the model checkpoint to a file
torch.save(model.state_dict(), "/content/drive/MyDrive/model_checkpoint_LSTM2.pth")

## Transformer with self attention

### model

In [79]:
class TotalEmbeddings(LightningModule):
    def __init__(self, dict_size, sizes, embedding_size, padding_idx, p=0.1):
        super().__init__()

        # Variables Assignment
        self.dict_size = dict_size
        self.max_length = max(sizes[0], sizes[1])
        self.embedding_size = embedding_size
        self.padding_idx = padding_idx
        self.dropout_probability = p

        self.scale = math.sqrt(self.embedding_size)

        # Layer Initialization
        self.embedding_layer = nn.Embedding(self.dict_size, self.embedding_size, padding_idx=self.padding_idx)
        self.dropout = nn.Dropout(p=self.dropout_probability)

    def forward(self, input):
        input = input.to(self.embedding_layer.weight.device)  # Ensure input is on the same device as the embedding layer
        char_embedding = self.embedding_layer(input) * self.scale

        # Sinusoidal Positional Embeddings Computation
        positional_embedding = self._initialize_pos_embedding(char_embedding.size(1)).to(char_embedding.device)

        return self.dropout(char_embedding + positional_embedding)

    def _initialize_pos_embedding(self, max_len):
        positional_embeddings = torch.zeros(max_len, self.embedding_size, device=self.embedding_layer.weight.device)

        for pos in range(max_len):
            for i in range(0, self.embedding_size, 2):
                positional_embeddings[pos, i] = math.sin(pos / (10000 ** (i / self.embedding_size)))
                positional_embeddings[pos, i + 1] = math.cos(pos / (10000 ** (i / self.embedding_size)))

        return positional_embeddings

In [80]:
class PositionwiseFFN(LightningModule):
    def __init__(self, embedding_size, hidden_units):
        super().__init__()

        # Variables Assignment
        self.embedding_size = embedding_size
        self.hidden_units = hidden_units

        # Layers Initialization
        self.linear_1 = nn.Linear(self.embedding_size, self.hidden_units)
        self.linear_2 = nn.Linear(self.hidden_units, self.embedding_size)
        self.relu = nn.ReLU()

    def forward(self, input):
        output = self.linear_1(input)  # shape (batch_size, input_len, hidden_units)
        output = self.relu(output)
        output = self.linear_2(output)  # shape (batch_size, input_len, embedding_size)
        return output




In [81]:
class Decoder(LightningModule):

  def __init__(self, embedding_size, hidden_units, num_layers,p_AT, p=0.1):
    super().__init__()

    # Variables Assignment
    self.embedding_size = embedding_size

    self.hidden_units = hidden_units
    self.num_layers = num_layers
    self.dropout_probability = p
    self.p_attention = p_AT

    # Layers Initialization
    self.layers = nn.ModuleList(
        [DecoderLayer(self.embedding_size,  self.hidden_units, p_AT= self.p_attention, p=self.dropout_probability) for _ in range(self.num_layers)])

  def forward(self, encoder_batch, outputs_batch, src_padding_mask, tgt_padding_mask):
    # encoder_batch shape (batch_size, encoder_len, embedding_size)
    # outputs_batch shape (batch_size, outputs_len, embedding_size)

    decoder_layer_ouput = outputs_batch

    for decoder_layer in self.layers:
      decoder_layer_ouput = decoder_layer(encoder_batch, decoder_layer_ouput, src_padding_mask, tgt_padding_mask)  # shape (batch_size, outputs_len, embedding_size)

    return decoder_layer_ouput  # shape (batch_size, outputs_len, embedding_size)

In [82]:
class DecoderLayer(LightningModule):

  def __init__(self, embedding_size,  hidden_units, p_AT, p=0.1):
    super().__init__()

    # Variables Assignment
    self.embedding_size = embedding_size
    self.hidden_units = hidden_units
    self.dropout_probability = p
    self.p_attention = p_AT
    # Layers Initialization
    self.ffn = PositionwiseFFN(self.embedding_size, self.hidden_units)

    self.selfAttn = SelfAttention(self.p_attention)

    self.layer_norm_1 = nn.LayerNorm(self.embedding_size, device=device)
    self.layer_norm_2 = nn.LayerNorm(self.embedding_size, device=device)
    self.layer_norm_3 = nn.LayerNorm(self.embedding_size, device=device)
    self.layer_norm_4 = nn.LayerNorm(self.embedding_size, device=device)
    self.densefilter = PositionwiseFeedforward(hid_dim=self.p_attention.d_x,
                                                   pf_dim=self.p_attention.d_r,
                                                   dropout=self.p_attention.dropout)
    self.dropout = nn.Dropout(p=self.dropout_probability)

  def forward(self, src, trg, src_mask, trg_mask):
    # encoder_batch shape (batch_size, encoder_len, embedding_size)
    # outputs_batch shape (batch_size, outputs_len, embedding_size)

    z = self.layer_norm_2(trg)
    z = self.selfAttn(z, z, z, trg_mask)
    z = self.dropout(z)
    trg = trg + z

    # Encoder attention
    z = self.layer_norm_2(trg)
    z = self.selfAttn(z, src, src, src_mask)
    z = self.dropout(z)
    trg = trg + z

    # Dense filter
    z = self.layer_norm_3(trg)
    z = self.densefilter(z)
    z = self.dropout(z)
    trg = trg + z

    return self.layer_norm_4(trg)

In [83]:
class Encoder(LightningModule):

  def __init__(self, embedding_size,  hidden_units, num_layers,p_AT, p=0.1):
    super().__init__()

    # Variables Assignment
    self.embedding_size = embedding_size
    self.hidden_units = hidden_units
    self.num_layers = num_layers
    self.dropout_probability = p
    self.p_attention = p_AT

    # Layers Initialization
    self.layers = nn.ModuleList(
        [EncoderLayer(self.embedding_size, self.hidden_units,self.p_attention, p=self.dropout_probability) for _ in range(self.num_layers)])

  def forward(self, batch, src_padding_mask):  # input shape (batch_size, input_len, embedding_size)

    encoder_layer_output = batch

    for encoder_layer in self.layers:
      encoder_layer_output = encoder_layer(encoder_layer_output, src_padding_mask)  # shape (batch_size, input_len, embedding_size)

    return encoder_layer_output  # shape (batch_size, input_len, embedding_size)

In [84]:
class EncoderLayer(LightningModule):

    def __init__(self, embedding_size, hidden_units,p_AT, p=0.1):
        super().__init__()

        # Variables Assignment
        self.embedding_size = embedding_size
        self.hidden_units = hidden_units
        self.dropout_probability = p
        self.p_attention = p_AT

        # Layers Initialization
        self.selfAttn = SelfAttention(self.p_attention)
        self.layer_norm_1 = nn.LayerNorm(self.embedding_size)
        self.dense_filter = PositionwiseFeedforward(self.embedding_size, self.hidden_units, self.dropout_probability)
        self.layer_norm_2 = nn.LayerNorm(self.embedding_size)
        self.dropout = nn.Dropout(p=self.dropout_probability)
        self.layer_norm_3 = nn.LayerNorm(self.embedding_size)
    def forward(self, src, src_mask):
        # src = [batch_size, src_seq_size, hid_dim]
        # src_mask = [batch_size, src_seq_size]

        # Sublayer 1
        z = self.layer_norm_1(src)
        z = self.selfAttn(z, z, z, src_mask)
        z = self.dropout(z)
        src = src + z

        # Sublayer 2
        z = self.layer_norm_2(src)
        z = self.dense_filter(z)
        z = self.dropout(z)
        src = src + z

        return self.layer_norm_3(src)

In [85]:
class PositionwiseFeedforward(LightningModule):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.pf_dim = pf_dim

        self.linear1 = nn.Linear(hid_dim, pf_dim)
        self.linear2 = nn.Linear(pf_dim, hid_dim)
        self.dropout = nn.Dropout(dropout)

        self.reset_parameters()

    def forward(self, x):
        x = self.linear1(x)
        x = self.dropout(F.relu(x))
        x = self.linear2(x)

        return x

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.linear1.weight)
        nn.init.xavier_uniform_(self.linear2.weight)

In [86]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import pytorch_lightning as pl

class SelfAttention(pl.LightningModule):
    def __init__(self, p):
        super().__init__()
        self.p = p
        self.d_h = p.d_x
        self.n_I = p.n_I

        self.W_q = nn.Linear(self.d_h, p.d_q * p.n_I)
        self.W_k = nn.Linear(self.d_h, p.d_k * p.n_I)
        self.W_v = nn.Linear(self.d_h, p.d_v * p.n_I)
        self.W_r = nn.Linear(self.d_h, p.d_r * p.n_I)

        self.W_o = nn.Linear(p.d_v * p.n_I, p.d_x)

        self.dropout = nn.Dropout(p.dropout)
        self.dot_scale = torch.FloatTensor([math.sqrt(p.d_k)])
        self.mul_scale = torch.FloatTensor([1.0 / math.sqrt(math.sqrt(2) - 1)])

    def forward(self, query, key, value, mask=None):
        # query = key = value = [batch_size, seq_len, hid_dim]
        bsz = query.shape[0]

        Q = self.W_q(query)
        K = self.W_k(key)
        V = self.W_v(value)
        R = self.W_r(query)

        Q = Q.view(bsz, -1, self.n_I, self.p.d_q).permute(0, 2, 1, 3)
        K = K.view(bsz, -1, self.n_I, self.p.d_k).permute(0, 2, 1, 3)
        V = V.view(bsz, -1, self.n_I, self.p.d_v).permute(0, 2, 1, 3)
        R = R.view(bsz, -1, self.n_I, self.p.d_r).permute(0, 2, 1, 3)
        dot = torch.einsum('bhid,bhjd->bhij', Q, K) / self.dot_scale.to(key.device)

        attention = self.dropout(F.softmax(dot, dim=-1))

        v_bar = torch.einsum('bhjd,bhij->bhid', V, attention)

        new_v = v_bar * R
        new_v = new_v.permute(0, 2, 1, 3).contiguous()

        new_v = new_v.view(bsz, -1, self.n_I * self.p.d_v)

        x = self.W_o(new_v)

        return x

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.W_q.weight)
        nn.init.xavier_uniform_(self.W_k.weight)
        nn.init.xavier_uniform_(self.W_v.weight)
        nn.init.xavier_uniform_(self.W_o.weight)
        nn.init.normal_(self.W_r.weight, mean=0, std=1.0 / math.sqrt(self.p.d_r))


In [87]:
class SelfAttentionConfig:
    def __init__(self, d_x, d_q, d_k, d_v, d_r, dropout, n_I):
        self.d_x = d_x
        self.d_q = d_q
        self.d_k = d_k
        self.d_v = d_v
        self.d_r = d_r
        self.dropout = dropout
        self.n_I = n_I
class TP_Transformer_SelfAttention(LightningModule):

  def __init__(self,char_to_idx,idx_to_char,embedding_size = 512, hidden_units = 2048, num_heads = 8, num_encoder_layers=6, num_decoder_layers= 6, padding_idx=0, p=0.):

    super().__init__()


    self.pad_token = "PAD"


    self.dict_size = len(char_to_idx)
    self.question_max_length = 68
    self.answer_max_length = 9
    self.padding_idx = padding_idx
    self.idx_to_char = idx_to_char
    self.dropout_probability = p
    self.batch_size = 128
    self.embedding_size = embedding_size
    self.hidden_units = hidden_units
    self.num_heads = num_heads
    self.num_encoder_layers = num_encoder_layers
    self.num_decoder_layers = num_decoder_layers
    self.p_attention = SelfAttentionConfig(d_x=512, d_q=64, d_k=64, d_v=64, d_r=64, dropout=self.dropout_probability, n_I=8)

    # Layers Initialization
    self.total_embedding = TotalEmbeddings(self.dict_size, [self.question_max_length, self.answer_max_length], self.embedding_size, self.padding_idx, p=self.dropout_probability)

    self.encoder = Encoder(self.embedding_size,  hidden_units=self.hidden_units, num_layers=self.num_encoder_layers, p=self.dropout_probability, p_AT=self.p_attention)

    self.decoder = Decoder(self.embedding_size,  hidden_units=self.hidden_units, num_layers=self.num_decoder_layers, p=self.dropout_probability, p_AT=self.p_attention)

  def forward(self, batch):


    questions = (batch[0]).to(device)  # shape (batch_size, question_max_length)
    answers = (batch[2]).to(device)  # shape (batch_size, answer_max_length-1)

    source_pad_mask = (questions == 0).to(device)  # shape (batch_size, question_max_length)
    target_pad_mask = (answers == 0).to(device)  # shape (batch_size, answer_max_length-1)

    embedded_questions = self.total_embedding(questions)  # shape (batch_size, question_max_length, embedding_size)
    embedded_answers = self.total_embedding(answers)  # shape (batch_size, answer_max_length-1, embedding_size)

    encoder_output = self.encoder(embedded_questions, src_padding_mask=source_pad_mask)  # shape (batch_size, question_max_length, embedding_size)

    decoder_output = self.decoder(encoder_output, embedded_answers, src_padding_mask=source_pad_mask, tgt_padding_mask=target_pad_mask)  # shape (batch_size, answer_max_length-1, embedding_size)

    # Softmax not used because of nn.CrossEntropyLoss
    return torch.matmul(decoder_output, torch.transpose(self.total_embedding.embedding_layer.weight, 0, 1))  # shape (batch_size, answer_max_length-1, dict_size)

  def training_step(self, batch, _):
    # Preparing inputs

    batch_answers = batch[2][:, 1:].flatten(0, 1)  # shape (batch_size * answer_max_length-1)
    batch[2] = batch[2][:, :-1]  # shape (batch_size, answer_max_length-1)

    # Computing prediction and loss
    pred = self(batch).flatten(0, 1)  # shape (batch_size * answer_max_length-1, dict_size)

    loss = F.cross_entropy(pred, batch_answers)
    #print("loss: ", loss)
    return loss

  def validation_step(self, batch, _):
    #input_ids, _, batch_answers, _ = batch
    batch_answers = batch[2]  # shape (batch_size, answer_max_length)

    # Computing prediction and accuracy
    pred = self.predict(batch)  # shape (batch_size, answer_max_length-1, dict_size)
    accuracy = paper_accuracy(pred, batch_answers)  # accuracy for the current batch as defined in the "mathematics dataset" paper

    print("val_tot_accuracy", accuracy)  # at the end of every epoch it logs the average of the accuracies of each batch
    return accuracy

  def test_step(self, batch, batch_idx):
    batch_answers = batch[2]  # shape (batch_size, answer_max_length)

    pred = self.predict(batch.copy())  # shape (batch_size, answer_max_length-1, dict_size)
    accuracy = paper_accuracy(pred, batch_answers)  # accuracy for the current batch as defined in the "mathematics dataset" paper

    print("test_tot_accuracy", accuracy)  # at the end of every epoch it is logged the average of the accuracies of each batch
    return accuracy

  def predict(self, batch):  # shape (batch_size, question_max_length)
    batch[2] = torch.tensor([[1] for j in range(len(batch[2]))], device=device).long()

    for i in range(self.answer_max_length-1):
      transformer_result = self(batch)
      batch[2] = torch.tensor([], device=device).long()

      for j in range(len(transformer_result)):
        predicted_chars = torch.argmax(transformer_result[j], 1)

        if len(predicted_chars.shape)==1:
          predicted_chars = predicted_chars.unsqueeze(0)

        start_line_char = torch.tensor([1], device=device).unsqueeze(0).long()
        predicted_chars = torch.cat((start_line_char, predicted_chars), 1)

        batch[2] = torch.cat((batch[2], predicted_chars), 0)

    return F.one_hot(batch[2], num_classes=self.dict_size)  # shape (batch_size, answer_max_length-1, dict_size)

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=1e-4, betas=(0.9, 0.995))




### training

In [88]:
# Initialize the model
model = TP_Transformer_SelfAttention(char_to_idx, idx_to_char)

In [89]:
import pytorch_lightning as pl
trainer = pl.Trainer(
                  # Set the number of GPUs to use (change to 0 if you want to run on CPU)
    max_epochs=1,   # Number of updates per epoch to display
)
# Train the model using the train_dataloaders
trainer.fit(model, train_dataloaders,test_dataloaders)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type            | Params
----------------------------------------------------
0 | total_embedding | TotalEmbeddings | 24.6 K
1 | encoder         | Encoder         | 20.5 M
2 | decoder         | Decoder         | 20.9 M
----------------------------------------------------
41.4 M    Trainable params
0         Non-trainable params
41.4 M    Total params
165.681   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

<ipython-input-55-2563b7be2ee6>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return [torch.tensor(self.input_ids[item], dtype=torch.long).to(device),
<ipython-input-55-2563b7be2ee6>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(self.answer_ids[item], dtype=torch.long).to(device),


prediction pre:  torch.Size([16, 9, 48])
target pre:  torch.Size([16, 9])
prediction:  torch.Size([16, 9])
target:  torch.Size([16, 9])
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
target:  tensor([ 3, 40, 30,  6, 33, 40,  0,  0,  0], device='cuda:0')
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
target:  tensor([ 3, 40, 23,  8, 10,  6, 33, 40,  0], device='cuda:0')
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
target:  tensor([ 3, 40, 23, 31, 44,  6, 33, 40,  0], device='cuda:0')
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
target:  tensor([ 3, 40, 23, 43, 44,  6, 33, 40,  0], device='cuda:0')
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
target:  tensor([ 3, 40, 23, 30,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
target:  tensor([ 3, 40,  8, 10,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], 

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

prediction pre:  torch.Size([16, 9, 48])
target pre:  torch.Size([16, 9])
prediction:  torch.Size([16, 9])
target:  torch.Size([16, 9])
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
target:  tensor([ 3, 40, 23,  8, 36,  6, 33, 40,  0], device='cuda:0')
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
target:  tensor([ 3, 40, 23, 31, 27,  6, 33, 40,  0], device='cuda:0')
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
target:  tensor([ 3, 40, 23, 43, 31,  6, 33, 40,  0], device='cuda:0')
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
target:  tensor([ 3, 40, 23,  8, 27,  6, 33, 40,  0], device='cuda:0')
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
target:  tensor([ 3, 40, 43, 36,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
target:  tensor([ 3, 40, 23, 31,  8,  6, 33, 40,  0], device='cuda:0')
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], 

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [90]:
import torch

# Assuming 'model' is your PyTorch model
# Save the model checkpoint to a file
torch.save(model.state_dict(), "/content/drive/MyDrive/model_checkpoint_selfAttn1.pth")

In [91]:
model.eval()
model = TP_Transformer_SelfAttention(char_to_idx, idx_to_char)

In [92]:
trainer.test(model, test_dataloaders)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

<ipython-input-55-2563b7be2ee6>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return [torch.tensor(self.input_ids[item], dtype=torch.long).to(device),
<ipython-input-55-2563b7be2ee6>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(self.answer_ids[item], dtype=torch.long).to(device),


prediction pre:  torch.Size([16, 9, 48])
target pre:  torch.Size([16, 9])
prediction:  torch.Size([16, 9])
target:  torch.Size([16, 9])
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
target:  tensor([ 3, 40, 31, 36,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
target:  tensor([ 3, 40, 43, 36,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
target:  tensor([ 3, 40, 36, 30,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
target:  tensor([ 3, 40, 36, 43,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
target:  tensor([ 3, 40, 43, 31,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
target:  tensor([ 3, 40, 31, 43,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], 

# Prediction

In [15]:
!gdown 1_EQ2dPfHBg-xV8qyYeGju9W3YE0tL1dN -O /content/char_to_idx

!gdown 1-0KbMACdGx7GrEXOL8Q07cER3znqcOpQ -O /content/idx_to_char
!gdown 1ASZ-9GYAsHOz60x2280VgUnumC8C5HgQ -O /content/tp_transformer
!gdown 1-6SMn3w6LgWppRLpWwp2V_Opxlf0BDvm -O /content/lstm_transformer
!gdown 1-IOVAF8xa8qy6H-KDvzAm1-ligamv5Ae -O /content/multihead_transformer

Downloading...
From: https://drive.google.com/uc?id=1_EQ2dPfHBg-xV8qyYeGju9W3YE0tL1dN
To: /content/char_to_idx
100% 316/316 [00:00<00:00, 1.43MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-0KbMACdGx7GrEXOL8Q07cER3znqcOpQ
To: /content/idx_to_char
100% 316/316 [00:00<00:00, 1.58MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=1ASZ-9GYAsHOz60x2280VgUnumC8C5HgQ
From (redirected): https://drive.google.com/uc?id=1ASZ-9GYAsHOz60x2280VgUnumC8C5HgQ&confirm=t&uuid=dd513d48-f821-439c-8b66-64e1143e42fc
To: /content/tp_transformer
100% 247M/247M [00:03<00:00, 67.4MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=1-6SMn3w6LgWppRLpWwp2V_Opxlf0BDvm
From (redirected): https://drive.google.com/uc?id=1-6SMn3w6LgWppRLpWwp2V_Opxlf0BDvm&confirm=t&uuid=e979e45e-dc42-4099-9718-d529a7750944
To: /content/lstm_transformer
100% 756M/756M [00:08<00:00, 85.3MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=1-IOVAF8xa8qy6H-KDvzAm1-ligamv5Ae
From 

In [17]:
# Load char_to_idx from the file
params = [None, None, 0, 0]
with open("/content/char_to_idx", 'rb') as file:
    params[0] = pickle.load(file)

# Load idx_to_char from the file
with open("/content/idx_to_char", 'rb') as file:
    params[1] = pickle.load(file)


params[2]=65
params[3]=7

In [14]:
#import pickle
#with open("/content/drive/MyDrive/char_to_idx_updated.pkl", 'wb') as file:
#    pickle.dump(char_to_idx, file)

# Save idx_to_char to a new pickle file
#with open("/content/drive/MyDrive/idx_to_char_updated.pkl", 'wb') as file:
#    pickle.dump(idx_to_char, file)

In [112]:
# Choose your model
choice = 1
trainer = pl.Trainer(max_epochs=10)
import pickle
if choice == 1:
    model = LSTMmodel(char_to_idx, idx_to_char)
    model.load_state_dict(torch.load("/content/lstm_transformer"), strict=False)
    model.eval()
    trainer.test(model, test_dataloaders)
elif choice == 2:
    model=TP_Transformer(char_to_idx, idx_to_char)
    model.load_state_dict(torch.load("/content/tp_transformer"), strict=False)
    model.eval()
    trainer.test(model, test_dataloaders)
elif choice == 3:
    model = TP_Transformer_SelfAttention(char_to_idx, idx_to_char)
    model.load_state_dict(torch.load("/content/multihead_transformer"), strict=False)
    model.eval()
    trainer.test(model, test_dataloaders)
else:
    print("No model selected")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

<ipython-input-55-2563b7be2ee6>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return [torch.tensor(self.input_ids[item], dtype=torch.long).to(device),
<ipython-input-55-2563b7be2ee6>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(self.answer_ids[item], dtype=torch.long).to(device),


prediction pre:  torch.Size([16, 9, 48])
target pre:  torch.Size([16, 9])
prediction:  torch.Size([16, 9])
target:  torch.Size([16, 9])
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 43, 36,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 23, 10,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40,  9,  6, 33, 40,  0,  0,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 23, 36, 36,  6, 33, 40,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 23,  9,  6, 33, 40,  0,  0], device='cuda:0')
predicted:  tensor([ 1, 40, 23,  6,  6, 33, 40,  0,  0], device='cuda:0')
target:  tensor([ 3, 40, 43, 31,  6, 33, 40,  0,  0], device='cud

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


# Prediction

Before running the code it is necessary to previously run the section where the selected model is defined

In [133]:
# Choose your model
choice = 1
trainer = pl.Trainer(max_epochs=10)
import pickle
if choice == 1:
    model = LSTMmodel(char_to_idx, idx_to_char)
    model.load_state_dict(torch.load("/content/drive/MyDrive/model_checkpoint_LSTM2.pth"), strict=False)
    model.eval()
elif choice == 2:
    model=TP_Transformer(char_to_idx, idx_to_char)
    model.load_state_dict(torch.load("/content/tp_transformer_checkpoint.pkl13"), strict=False)
    model.eval()
elif choice == 3:
    model = TP_Transformer_SelfAttention(char_to_idx, idx_to_char)
    model.load_state_dict(torch.load("/content/transformer_our_model.ckpt3"), strict=False)
    model.eval()
else:
    print("No model selected")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


### Prediction functions


In [134]:
def tokenize( input,char_to_idx,question_max_length):
  question = []
  question.append(1)  # adding start of line char

  for char in input:
    if char not in char_to_idx.keys():
      question.append(1)
    else:
      question.append(char_to_idx[char])  # turning character into index

  question.append(2)  # adding end of line char

  for i in range(question_max_length-len(input)-2):
      question.append(0)  # adding padding after

  question = torch.tensor(question, device=device).unsqueeze(0)

  return {"question": question}

In [135]:
import numpy as np

def create_attention_mask(input_tensor):
    """
    Create a fully connected attention mask for a given tensor.

    Args:
        input_tensor (numpy.ndarray): The input tensor.

    Returns:
        numpy.ndarray: The attention mask with 1s indicating positions to attend to.
    """
    N, M = input_tensor.shape
    attention_mask = np.ones((N, M), dtype=int)
    return attention_mask


In [136]:
def encode_question(inputs1,inputs2,inputs3,inputs4):

    tesst = Dataset_class3(inputs1,inputs2,inputs3,inputs4)
    dataloaders = torch.utils.data.DataLoader(tesst, shuffle=False, batch_size=16)

    for batch in dataloaders:
      pred = model.predict(batch.copy())
    predicted = torch.argmax(pred, dim=2)

    return predicted


In [137]:
def tensor_to_string(tensor, idx_to_char):
    # Extract the elements from the tensor and map them to characters
    char_list = [idx_to_char[idx.item()] for idx in tensor.squeeze()]
    # Join the characters to form the string
    return ''.join(char_list)




In [138]:
class Dataset_class3:
  def __init__(self, inputs1,inputs2,inputs3,inputs4):
    self.input_ids = inputs1
    self.attention_mask = inputs2
    self.answer_ids = inputs3
    self.answer_attention_mask = inputs4

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, item):
    return [torch.tensor(self.input_ids[item], dtype=torch.long),
            torch.tensor(self.attention_mask[item], dtype=torch.long),
            torch.tensor(self.answer_ids[item], dtype=torch.long),
            torch.tensor(self.answer_attention_mask[item], dtype=torch.long)]

In [139]:
def answerquestion(question):


  string_question = tokenize( question,params[0],params[2])
  string_answer = tokenize( "b'-19\n'",params[0],params[3])
  TensorPredictedAnswer = encode_question(string_question["question"],create_attention_mask(string_question["question"]),string_answer["question"],create_attention_mask(string_answer["question"]))
  decoded_answer = tensor_to_string(TensorPredictedAnswer, params[1])
  return decoded_answer

In [140]:
Example_inthe_dataset = 22
test = mathdataset["test"][Example_inthe_dataset]

In [141]:
params[0]

{' ': 0,
 'g': 1,
 'S': 2,
 'b': 3,
 'j': 4,
 'f': 5,
 '\\': 6,
 'm': 7,
 '1': 8,
 '6': 9,
 '7': 10,
 'v': 11,
 '.': 12,
 'd': 13,
 '5': 14,
 'p': 15,
 '=': 16,
 's': 17,
 'w': 18,
 'x': 19,
 'a': 20,
 'y': 21,
 'i': 22,
 '-': 23,
 'k': 24,
 'e': 25,
 't': 26,
 '0': 27,
 'l': 28,
 '+': 29,
 '9': 30,
 '3': 31,
 'r': 32,
 'n': 33,
 'z': 34,
 'c': 35,
 '4': 36,
 'q': 37,
 'h': 38,
 '*': 39,
 "'": 40,
 'u': 41,
 'o': 42,
 '2': 43,
 '8': 44,
 '<start>': 45,
 '<end>': 46,
 'PAD': 47}

In [142]:
string_question = tokenize( test["question"],params[0],params[2])
string_answer = tokenize( test["answer"],params[0],params[3])


In [148]:
string_question["question"]=string_question["question"].to(device)
string_answer["question"]=string_question["question"].to(device)

In [149]:
TensorPredictedAnswer = encode_question(string_question["question"],create_attention_mask(string_question["question"]),string_answer["question"],create_attention_mask(string_answer["question"]))

<ipython-input-138-de5139cfeb82>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return [torch.tensor(self.input_ids[item], dtype=torch.long),
<ipython-input-138-de5139cfeb82>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(self.answer_ids[item], dtype=torch.long),


In [150]:
# Translate the tensors to strings
decoded_answer = tensor_to_string(TensorPredictedAnswer, params[1])

### Preditction

In [151]:
print("Question:",mathdataset["test"]["question"][Example_inthe_dataset])
print("Real answer:",mathdataset["test"]["answer"][Example_inthe_dataset])
print("Predicted answer:",decoded_answer)

Question: b'Solve 56*u - 1677848 = -1678912 for u.\n'
Real answer: b'-19\n'
Predicted answer: g''-\\n' 


In [152]:
user_question = input("Type equation: ")
variable = input("For which varible?: ")
user_question_modifed = "b'Solve "+user_question+" for "+variable+ ".\n'"

answer = answerquestion(user_question_modifed)
print("The answer to your question is",answer)

Type equation: 2+3 = a
For which varible?: a


<ipython-input-138-de5139cfeb82>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return [torch.tensor(self.input_ids[item], dtype=torch.long),
<ipython-input-138-de5139cfeb82>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(self.answer_ids[item], dtype=torch.long),


The answer to your question is g''-\\n' 
